In [8]:
import re
import torch
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
from bertopic import BERTopic
import demoji
import spacy
import unidecode
from umap import UMAP
from bertopic import BERTopic
import gensim
import gensim.corpora as corpora
from sentence_transformers import SentenceTransformer
from hdbscan import HDBSCAN
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from bertopic.vectorizers import ClassTfidfTransformer
from gensim.models.coherencemodel import CoherenceModel

In [9]:
import torch


In [10]:
if torch.cuda.is_available():
    print("Using GPU.")
else:
    print("Using CPU.")


Using CPU.


In [11]:
nlp = spacy.load("es_core_news_sm")

In [12]:
nltk.download('stopwords')
nltk.download('punkt')
stop_words = stopwords.words('spanish')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\CARG\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\CARG\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [13]:
spanish_stop_words = nltk.corpus.stopwords.words('spanish')

**Filtro por mes**

In [14]:
df = pd.read_csv(r"C:\Users\CARG\Desktop\Codigos_tesis_bertopic\medio_año_2022_politica.csv") 

In [15]:
df.head(1)

,date_source,text,topic_label
0,2022-03-01 00:00:01+00:00,@R77684031Emilio @RickyVzla3 🙊🙈🙈🤣🤣🤣🤣🤣Es cierto,politica


**Filtro por mes**

In [16]:
df['date_source'] = pd.to_datetime(df['date_source'])

In [17]:
df_prueba = df[df['date_source'].dt.month.isin([6])].copy()


In [18]:
num_filas = df_prueba.shape[0]

# Imprimir el número de filas
print("El dataframe df_prueba tiene", num_filas, "filas.")

El dataframe df_prueba tiene 361315 filas.


***Limpieza de Twitter***

Dejamos @? me parece importante

 @MiguelPoloP Ud hablando de una población que ni representa?


In [19]:
#eliminacion de emojis
def clean_text(x):
  x = str(x)
  x = x.lower()
  x = re.sub(r'#[A-Za-z0-9]*', ' ', x)
  x = re.sub(r'https*://.*', ' ', x)
  

  x = re.sub(r'@[A-Za-z0-9]+', ' ', x)
  
  # remove emojis
  demoji.download_codes()
  x = demoji.replace(x, '')

  # remove accents
  x = unidecode.unidecode(x)
  
  tokens = word_tokenize(x)
  x = ' '.join([w for w in tokens if not w.lower() in stop_words])
  x = re.sub(r'[%s]' % re.escape('!"#$%&\()*+,-./:;<=>?@[\\]^_`{|}~“…”’'), ' ', x)
  x = re.sub(r'\d+', ' ', x)
  x = re.sub(r'\n+', ' ', x)
  x = re.sub(r'\s{2,}', ' ', x)
  return x

In [20]:
oracion = "@MiguelPoloP Ud hablando de una población que ni representa :# :) ?"
oracion_procesada = clean_text(oracion)
print(oracion_procesada)

ud hablando poblacion representa 


***Lematizador***

In [21]:
def lemmatize_text(text):
    doc = nlp(text)
    return " ".join([token.lemma_ for token in doc])


In [22]:
# Lematización de la oración de prueba
lemmatized_text = lemmatize_text(oracion_procesada)
# Impresión del resultado
print(lemmatized_text)

ud hablar poblacion representar


In [23]:
df_prueba['clean_text'] = df_prueba.text.apply(clean_text)
df_prueba.head()

,date_source,text,topic_label,clean_text
839409,2022-06-01 00:00:06+00:00,@Danielbricen Así es. Petro debatio con la bes...,politica,asi petro debatio bestia fico sucio quemado de...
839411,2022-06-01 00:00:20+00:00,"@isaiasim Te felicito, todo es voluntad. La cu...",politica,felicito voluntad
839412,2022-06-01 00:00:22+00:00,@Mijagui @medicenmarrr @sergio_fajardo Son tan...,politica,fajardo tan funestas alianzas bandidos roy be...
839413,2022-06-01 00:00:22+00:00,@sergio_fajardo Se quedó un caballero para hac...,politica,fajardo quedo caballero hacer politica sergio...
839414,2022-06-01 00:00:24+00:00,@Eganbernal Usted es un campesino iletrado arr...,politica,usted campesino iletrado arrastrado vivia pidi...


In [24]:
# Lematización de la columna 'clean_text' del dataframe df_prueba
df_prueba['lemmatized_text'] = df_prueba['clean_text'].apply(lemmatize_text)

In [25]:
df_prueba.head()

,date_source,text,topic_label,clean_text,lemmatized_text
839409,2022-06-01 00:00:06+00:00,@Danielbricen Así es. Petro debatio con la bes...,politica,asi petro debatio bestia fico sucio quemado de...,asi petro debatio bestia fico sucio quemado de...
839411,2022-06-01 00:00:20+00:00,"@isaiasim Te felicito, todo es voluntad. La cu...",politica,felicito voluntad,felicito voluntad
839412,2022-06-01 00:00:22+00:00,@Mijagui @medicenmarrr @sergio_fajardo Son tan...,politica,fajardo tan funestas alianzas bandidos roy be...,fajardo tanto funesto alianza bandido roy be...
839413,2022-06-01 00:00:22+00:00,@sergio_fajardo Se quedó un caballero para hac...,politica,fajardo quedo caballero hacer politica sergio...,fajardo quedo caballero hacer politico sergi...
839414,2022-06-01 00:00:24+00:00,@Eganbernal Usted es un campesino iletrado arr...,politica,usted campesino iletrado arrastrado vivia pidi...,usted campesino iletrado arrastrado vivia pedi...


In [26]:
timestamps = df_prueba.date_source.to_list()
tweets = df_prueba.lemmatized_text.to_list()

In [27]:
import os

os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [28]:
from sentence_transformers import SentenceTransformer
from umap import UMAP
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import silhouette_score, calinski_harabasz_score
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import fetch_20newsgroups
from sklearn.metrics import silhouette_score, calinski_harabasz_score
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import fetch_20newsgroups
from gensim.models import CoherenceModel
from gensim import corpora
import hdbscan
import pandas as pd

In [29]:
n_neighbors_list = [30,40,50]  # Valores para n_neighbors
n_components_list = [2,5,8]  # Valores para n_components
min_dist_list = [0.0, 0.1]  # Valores para min_dist

for n_neighbors in n_neighbors_list:
    for n_components in n_components_list:
        for min_dist in min_dist_list:
            umap_model = UMAP(n_neighbors=n_neighbors, n_components=n_components, min_dist=min_dist, metric='cosine', random_state=42)
            
            hdbscan_model = hdbscan.HDBSCAN(min_cluster_size=11, metric='euclidean', cluster_selection_method='eom', prediction_data=True)
            vectorizer_model = CountVectorizer(ngram_range=(1, 3), stop_words=spanish_stop_words)
            ctfidf_model = ClassTfidfTransformer()
            
            topic_model = BERTopic(
                verbose=False,
                embedding_model="paraphrase-multilingual-MiniLM-L12-v2",
                language="Spanish",
                umap_model=umap_model,
                hdbscan_model=hdbscan_model,
                vectorizer_model=vectorizer_model,
                ctfidf_model=ctfidf_model,
                nr_topics="auto",
                n_gram_range=(1, 3),
                calculate_probabilities=False
            )
            
            topics, probs = topic_model.fit_transform(tweets)
            filtered_text = df_prueba['lemmatized_text']
            
            documents = pd.DataFrame({"Document": filtered_text,
                                      "ID": range(len(filtered_text)),
                                      "Topic": topics})
            documents_per_topic = documents.groupby(['Topic'], as_index=False).agg({'Document': ' '.join})
            cleaned_docs = topic_model._preprocess_text(documents_per_topic.Document.values)
            
            # Extract vectorizer and analyzer from BERTopic
            vectorizer = topic_model.vectorizer_model
            analyzer = vectorizer.build_analyzer()
            
            # Extract features for Topic Coherence evaluation
            words = vectorizer.get_feature_names_out()
            tokens = [analyzer(doc) for doc in cleaned_docs]
            dictionary = corpora.Dictionary(tokens)
            corpus = [dictionary.doc2bow(token) for token in tokens]
            topic_words = [[words for words, _ in topic_model.get_topic(topic)]
                           for topic in range(len(set(topics))-1)]
            
            # Evaluate C_V
            coherence_model_cv = CoherenceModel(topics=topic_words,
                                                texts=tokens,
                                                corpus=corpus,
                                                dictionary=dictionary,
                                                coherence='c_v')
            coherence_cv = coherence_model_cv.get_coherence()
            
            print(f"CV (n_neighbors={n_neighbors}, n_components={n_components}, min_dist={min_dist}): {coherence_cv}")
            
            # Evaluate U_MASS
            coherence_model_umass = CoherenceModel(topics=topic_words,
                                                   texts=tokens,
                                                   corpus=corpus,
                                                   dictionary=dictionary,
                                                   coherence='u_mass')
            coherence_umass = coherence_model_umass.get_coherence()
            
            print(f"U_MASS (n_neighbors={n_neighbors}, n_components={n_components}, min_dist={min_dist}): {coherence_umass}")

ValueError: unable to interpret topic as either a list of tokens or a list of ids